In [1]:
from pathlib import Path
import sys
import time
import threading

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
%matplotlib inline
import autoclasswrapper as wrapper
import os 

afterall = {}
subfeatures = ['Total_Backward_Packets', 'Total_Length_of_Fwd_Packets']
os.environ['PATH'] += ":/home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c"
!touch /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
!echo "xref_class_report_att_list = 0, 1 \nreport_mode = \"data\" \ncomment_data_headers_p = true " >> /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
for feature in subfeatures:
    runtimeDir = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/" + feature
    if not Path(runtimeDir).exists():
        os.mkdir(runtimeDir)
    os.chdir(runtimeDir)
    
            # Create object to prepare dataset.
    clust = wrapper.Input()

    # Load datasets from tsv files.
    pathData = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/" + feature + ".tsv"
    clust.add_input_data(pathData, "real location")
    
    # Prepare input data:
    # - create a final dataframe
    # - merge datasets if multiple inputs
    clust.prepare_input_data()

    # Create files needed by AutoClass.
    clust.create_db2_file()
    clust.create_hd2_file()
    clust.create_model_file()
    clust.create_sparams_file(max_duration = 0)
    clust.create_rparams_file()
    !rm autoclass.r-params
    !cp /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params .
    
    !rm -f autoclass-run-* *.results-bin


    # Search autoclass in path.
    wrapper.search_autoclass_in_path()

    # Create object to run AutoClass.
    run = wrapper.Run()

    # Prepare run script.
    run.create_run_file()

    # Run AutoClass.
    run.run()
        
    timer = 0
    step = 2
    while not Path(runtimeDir + "/autoclass-run-success").exists():
        timer += step
        sys.stdout.write("\r")
        sys.stdout.write(f"Time: {timer} sec.")
        sys.stdout.flush()
        time.sleep(step)
        if Path("autoclass-run-failure").exists():
            print("error")
            break
             
    if Path(runtimeDir + "/autoclass-run-success").exists():
        results = wrapper.Output()
        results.extract_results()
        results.aggregate_input_data()
#         results.write_cdt()
#         results.write_cdt(with_proba=True)
#         results.write_class_stats()
#         results.write_dendrogram()
        result = pd.read_csv(runtimeDir + '/autoclass_out.tsv', encoding="utf-8", header=0, sep='\t', index_col=0)
        label =  pd.read_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/ProtocolName.tsv', encoding='utf-8', header=0, sep='\t', index_col=0)

        label['cluster'] = result['main-class']
        label.to_csv('app.csv', index = True, header=True)


        listH = []
        listApp = []
        listCluster = []
        def fun(x):
            listCluster.append(x['cluster'].values[0])
            listH.append(x['ProtocolName'].value_counts().max()/x.shape[0])
            listApp.append(x['ProtocolName'].value_counts().idxmax())


        label.groupby('cluster').apply(fun)

        H = sum(listH)/len(listH)
        print(listH)
        print(listApp)
        print(listCluster)
        print(H)

        dict = {'Cluster': listCluster, 'App': listApp, 'H': listH}

        saveResult = pd.DataFrame(dict)

        saveResult.to_csv(runtimeDir + '/resultCal.csv', index = True, header=True)
        
        afterall.update({feature:H})
        
round1 = pd.DataFrame(afterall)
round1.to_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/resl2.csv', index = True, header=True)

2020-04-26 21:29:25 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/Total_Backward_Packets.tsv' as 'real location' with error 0.01
2020-04-26 21:29:25 INFO     Detected encoding: ascii
2020-04-26 21:29:25 INFO     Found 4916 rows and 2 columns
2020-04-26 21:29:25 DEBUG    Checking column names
2020-04-26 21:29:25 DEBUG    Index name 'index'
2020-04-26 21:29:25 DEBUG    Column name 'Total_Backward_Packets'
2020-04-26 21:29:25 INFO     Checking data format
2020-04-26 21:29:25 INFO     Column 'Total_Backward_Packets'
2020-04-26 21:29:25 INFO     count      4916.000000
2020-04-26 21:29:25 INFO     mean        148.477828
2020-04-26 21:29:25 INFO     std        2089.554111
2020-04-26 21:29:25 INFO     min           0.000000
2020-04-26 21:29:25 INFO     50%           5.000000
2020-04-26 21:29:25 INFO     max      134496.000000
2020-04-26 21:29:25 INFO     ---
2020-04-26 21:29:25 INFO     Final dataframe has 4916 lines and 2 columns
2020-04-26 

Time: 720 sec.

2020-04-26 21:41:26 INFO     Extracting autoclass results
2020-04-26 21:41:26 INFO     Found 4916 cases classified in 57 classes
2020-04-26 21:41:30 INFO     Aggregating input data
2020-04-26 21:41:30 INFO     Writing classes + probabilities .tsv file
2020-04-26 21:41:30 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/Total_Length_of_Fwd_Packets.tsv' as 'real location' with error 0.01
2020-04-26 21:41:30 INFO     Detected encoding: ascii
2020-04-26 21:41:30 INFO     Found 4916 rows and 2 columns
2020-04-26 21:41:30 DEBUG    Checking column names
2020-04-26 21:41:30 DEBUG    Index name 'index'
2020-04-26 21:41:30 DEBUG    Column name 'Total_Length_of_Fwd_Packets'
2020-04-26 21:41:30 INFO     Checking data format
2020-04-26 21:41:30 INFO     Column 'Total_Length_of_Fwd_Packets'
2020-04-26 21:41:30 INFO     count    4.916000e+03
2020-04-26 21:41:30 INFO     mean     6.675576e+04
2020-04-26 21:41:30 INFO     std      6.491123e+05
2020-04-26

[0.09848484848484848, 0.036753445635528334, 0.08771929824561403, 0.17664670658682635, 0.06432748538011696, 0.0830860534124629, 0.14375, 0.08112874779541446, 0.3611111111111111, 0.15517241379310345, 0.0784313725490196, 0.07608695652173914, 0.10606060606060606, 0.3728813559322034, 0.13793103448275862, 0.08333333333333333, 0.05660377358490566, 0.09259259259259259, 0.08333333333333333, 0.34210526315789475, 0.10256410256410256, 0.09302325581395349, 0.18604651162790697, 0.06369426751592357, 0.13953488372093023, 0.48936170212765956, 0.10526315789473684, 0.14705882352941177, 0.11764705882352941, 0.15625, 0.15151515151515152, 0.1111111111111111, 0.13592233009708737, 0.1724137931034483, 0.15384615384615385, 0.19230769230769232, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.12, 0.125, 0.15, 0.2, 0.6363636363636364, 0.35294117647058826, 0.4666666666666667, 0.11764705882352941, 0.125, 0.125, 0.11764705882352941, 0.43478260869565216, 0.14285714285714285, 0.2857142857142857, 0.1, 0

2020-04-26 21:41:31 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-26 21:41:31 INFO     Writing run file
2020-04-26 21:41:31 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-26 21:41:31 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-26 21:41:31 INFO     Running clustering...


Time: 1452 sec.

2020-04-26 22:05:44 INFO     Extracting autoclass results
2020-04-26 22:05:45 INFO     Found 4916 cases classified in 53 classes
2020-04-26 22:05:48 INFO     Aggregating input data
2020-04-26 22:05:48 INFO     Writing classes + probabilities .tsv file


[0.10615711252653928, 0.09829059829059829, 0.06569343065693431, 0.10869565217391304, 0.12121212121212122, 0.057291666666666664, 0.06358381502890173, 0.1282051282051282, 0.06451612903225806, 0.0718954248366013, 0.1111111111111111, 0.5391304347826087, 0.12, 0.13157894736842105, 0.12345679012345678, 0.5056179775280899, 0.15384615384615385, 0.725, 0.1935483870967742, 0.0625, 0.140625, 0.11538461538461539, 0.5735294117647058, 0.20987654320987653, 0.14814814814814814, 0.12, 0.16981132075471697, 0.21818181818181817, 0.4897959183673469, 0.3111111111111111, 0.09090909090909091, 0.3877551020408163, 0.2413793103448276, 1.0, 1.0, 1.0, 0.24324324324324326, 0.36585365853658536, 0.1111111111111111, 0.25, 0.4827586206896552, 0.21212121212121213, 1.0, 0.12903225806451613, 0.1, 0.21052631578947367, 0.1, 0.16666666666666666, 0.5, 0.9230769230769231, 0.5384615384615384, 0.1111111111111111, 1.0]
['IP_ICMP', 'INSTAGRAM', 'HTTP', 'TELEGRAM', 'HTTP', 'MS_ONE_DRIVE', 'SKYPE', 'HTTP_DOWNLOAD', 'SPOTIFY', 'SSH',

ValueError: If using all scalar values, you must pass an index